# Azure Sentinel: Learn the KQL you need for Azure Sentinel 

## This Jupyter Notebook is a KQL study material extracted from a webinar by [@Ofer_Shezaf](https://techcommunity.microsoft.com/t5/user/viewprofilepage/user-id/293879) entitled **Azure Sentinel: Learn the KQL you need for Azure Sentinel** This is part of a series of Webinars on [**Become an Azure Sentinel Ninja**](https://techcommunity.microsoft.com/t5/azure-sentinel/become-an-azure-sentinel-ninja-the-complete-level-400-training/ba-p/1246310).

## Installing the library to perform KQL in Jupyter notebook

In [ ]:
!pip3 install Kqlmagic --no-cache-dir --upgrade

## Loading the KQLmagic library

In [ ]:
%reload_ext Kqlmagic

## Configuring some variables to facilitate the visualization

In [ ]:
%config Kqlmagic.auto_dataframe=False
%config Kqlmagic.auto_popup_schema=False
%config Kqlmagic.display_limit=10

## Loading other libraries for better visualization of results 

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = False

## Connecting to the Demo Log Analytics

In [ ]:
%kql loganalytics://workspace='DEMO_WORKSPACE';appkey='DEMO_KEY'

<h2 align='center'>===========================================================================<br>===========================================================================<br>HERE BEGINS KQL QUERIES EXTRACTED FROM THE WEBINAR</h2>


## 'where' operator

In [ ]:
%kql SecurityEvent | where * contains "Kusto"

In [ ]:
%kql SecurityEvent| where TimeGenerated> ago(1d)

In [ ]:
%kql SecurityEvent| where TimeGenerated> ago(1h) and EventID== 4624

In [ ]:
%kql SecurityEvent| where TimeGenerated > ago(1h)| where EventID == 4624| where AccountType =~ "user"

In [ ]:
%kql SecurityEvent | where EventID in (4624, 4625)

In [ ]:
%kql VMConnection | where ipv4_is_match (DestinationIp, "10.0.0.0/8")

## 'search' operator

In [ ]:
%kql search "10.1.5.5"

In [ ]:
%kql SecurityEvent | where TimeGenerated >= ago(1d) | search "Guest"

## 'extend' operator

In [ ]:
%kql SecurityEvent | extend ComputerNameLength = strlen(Computer)

In [ ]:
%%kql Perf
|where CounterName == "Free Megabytes"
|where InstanceName == "C:"
|extend FreeKB = CounterValue * 1000
|extend FreeGB = CounterValue / 1000

In [ ]:
%%kql SecurityEvent
| where EventID in ( 4624 , 4625 )
| extend rgroup = extract("resourcegroups/(.*)/providers",1,_ResourceId)

In [ ]:
%%kql SecurityEvent
| where EventID in ( 4624 , 4625 )
| extend rgroup = split(_ResourceId,"/",4)[0]

In [ ]:
%%kql SecurityEvent
| where EventID in ( 4624 , 4625 )
| parse _ResourceId with "/subscription/" sub "/resourcegroups/" rgroup "/providers" *

## A real-world example: TOR usage detection
from: https://github.com/Azure/Azure-Sentinel/blob/master/Detections/Syslog/squid_tor_proxies.yaml

In [ ]:
%%kql 
let timeframe = 1d;
let DomainList = dynamic(["tor2web.org", "tor2web.com", "torlink.co", "onion.to", "onion.ink", "onion.cab", "onion.nu", "onion.link", 
  "onion.it", "onion.city", "onion.direct", "onion.top", "onion.casa", "onion.plus", "onion.rip", "onion.dog", "tor2web.fi", 
  "tor2web.blutmagie.de", "onion.sh", "onion.lu", "onion.pet", "t2w.pw", "tor2web.ae.org", "tor2web.io", "tor2web.xyz", "onion.lt", 
  "s1.tor-gateways.de", "s2.tor-gateways.de", "s3.tor-gateways.de", "s4.tor-gateways.de", "s5.tor-gateways.de", "hiddenservice.net"]); 
//
Syslog
| where TimeGenerated >= ago(timeframe)
| where ProcessName contains "squid" 
| extend HTTP_Status_Code = extract("(TCP_(([A-Z]+)(_[A-Z]+)*)|UDP_(([A-Z]+)(_[A-Z]+)*))/([0-9]{3})",8,SyslogMessage),Domain = extract("(([A-Z]+ [a-z]{4,5}:\\/\\/)|[A-Z]+ )([^ :\\/]*)",3,SyslogMessage)
| where HTTP_Status_Code == "200"
| where Domain contains "."
| where Domain has_any (DomainList)

## Lab #1: Filtering
Find all Windows logon events starting 2 weeks ago until 1 week ago that occured on a computer with name which starts with "C".

In [ ]:
%%kql 
SecurityEvent
| where TimeGenerated between (ago(14d) .. ago(7d))
| where EventID == 4624 //"An account was successfully logged on" more info at https://www.ultimatewindowssecurity.com/securitylog/encyclopedia/event.aspx?eventID=4624
| where Computer startswith "C"

## 'summarize' command

In [ ]:
%kql SecurityEvent | summarize count() by Computer

In [ ]:
%%kql
WindowsFirewall
| where CommunicationDirection == "SEND" 
| where FirewallAction == "ALLOW"
| summarize dcount(SourceIP)

In [ ]:
%%kql
SecurityEvent
| where TimeGenerated > ago(1h)
| where EventID == 4624
| summarize count() by AccountType, Computer

## Lab #2: analysis
Find how many times each process ran per computer. Hints and guideline: (1) Event 4688 logs process creation; (2) which field represent the processes created and which the computer on which it was ran?

In [ ]:
%%kql
SecurityEvent
| where EventID == 4688 //"A new process has been created"
| summarize count() by NewProcessName, Computer

## Variants and add-ons to summarize

In [ ]:
%kql SecurityEvent | distinct Computer, Account

In [ ]:
%kql SecurityEvent | where EventID == 4624 | count

In [ ]:
%kql SecurityEvent | where EventID == 4624 | order by Account

In [ ]:
%kql SecurityEvent | top 10 by TimeGenerated desc

## 'order by' exercise

In [ ]:
%%kql
SecurityAlert
| where TimeGenerated > ago(7d) 
| extend severityOrder = case (
    AlertSeverity == "High", 3, 
    AlertSeverity == "Medium", 2, 
    AlertSeverity == "Low", 1, 
    AlertSeverity == "Informational", 0, 
    -1)
| order by severityOrder
| project-away severityOrder

## 'summarize' as filter: arg_min(), arg_max()

In [ ]:
%%kql
WindowsFirewall
| where TimeGenerated > ago(7d)
| summarize arg_max(TimeGenerated, *) by SourceIP

## Quiz #1

What is the difference between the following queries?

```
 SecurityEvent
| summarize arg_max(TimeGenerated, *) by Account 
| where EventID == "4624"
| count
```
and 
```
SecurityEvent
| where EventID == "4624"
| summarize arg_max(TimeGenerated, *) by Account 
| count
```
 

In [ ]:
%%kql SecurityEvent
| summarize arg_max(TimeGenerated, *) by Account 
| where EventID == "4624"
| count

In [ ]:
%%kql SecurityEvent
| where EventID == "4624"
| summarize arg_max(TimeGenerated, *) by Account 
| count

**ANSWER:** The line 2 filters the last TimeGenerated per Account and then from the remaining lines it tries to get only the event 4624. The problem is that not all last operation of each account was the 4624. The second kql query gets the last time that an Account was associated to EventID 4624.

## Lab #3: analysis
Find how source IPs from which traffic has been dropped on Windows machines more than 10 times in the last 7 days.

Hints and guideline:
- Connections to Windows machines are collected in the "WindowsFirewall"
- What is the name of the field which specifies traffic direction and can help determine inbound connections?
- What is the name of the field which specifies the firewall action and can help identify dropped connections?

In [ ]:
%%kql WindowsFirewall
|where TimeGenerated >= ago(7d)
|where CommunicationDirection == 'RECEIVE'
|where FirewallAction == 'DROP'
|summarize c=count() by SourceIP
|where c>=10 

## A real-world example: password spray detection
from: https://github.com/Azure/Azure-Sentinel/blob/master/Detections/SigninLogs/DisabledAccountSigninsAcrossManyApplications.yaml

In [ ]:
%%kql 
let timeframe = 1d;
let threshold = 3;
SigninLogs
| where TimeGenerated >= ago(timeframe) 
| where ResultType == "50057"
| where ResultDescription =~ "User account is disabled. The account has been disabled by an administrator."
| summarize applicationCount = dcount(AppDisplayName) by UserPrincipalName, IPAddress
| where applicationCount >= threshold

# Prepare

## 'project' operator

In [ ]:
%kql SecurityEvent | project TimeGenerated, Computer

In [ ]:
%kql SecurityEvent | project IsImportant = iff(Computer contains "CEO", true, false)

## 'summarize' to prepare: make_list(), make_set()

In [ ]:
%kql SecurityEvent | summarize make_set(Account) by Computer

# Visualize

## 'summarize': bin and time series

In [ ]:
%%kql SecurityEvent
|summarize count() by bin(TimeGenerated, 1h)
|render timechart

In [ ]:
%%kql SecurityEvent 
| where TimeGenerated > ago(7d) 
| summarize count() by bin(TimeGenerated, 1d)
| render timechart

In [ ]:
%%kql VMConnection
| summarize count() by SourceIp 
| sort by count_ desc 
| render barchart

## Lab #4: visualization

Chart the rate of process creation on all domain controllers.

Hints and guideline:
- Process creation is Windows event 4688
- Domain controller names start with "DC"
- Create multiple charts by aggregating additional more than one field

In [ ]:
%%kql SecurityEvent
| where Computer startswith "DC"
| where EventID == "4688"
| summarize count() by Computer, bin(TimeGenerated, 1h) 
| render timechart

## Unrelated 'countif' operator example 

In [ ]:
%%kql 
SecurityEvent 
| summarize AdminSuccessfullLogons = countif(Account contains "Admin" and EventID == 4624), 
            AdminFailedLogons = countif(Account contains "Admin" and EventID == 4625)

## Lab #5: visualization

Render graph of successful vs failed logons over the last 7days, use alias for the legend ("Success", "Failed")

Hints and guideline:
- In the Security Event table, successful logon has eventid 4624, and failed logon event id 4625.
- Check if aggregation function countif can help.

In [ ]:
%%kql
SecurityEvent 
| summarize Success=countif(EventID == 4624), 
            Failed=countif(EventID == 4625) by bin(TimeGenerated, 1h)
| render timechart

# Advanced Topics

## 'let' statement: declare and reuse variables

In [ ]:
%%kql 
let timeOffset = 7d;
let discardEventId = 4688;
SecurityEvent
| where TimeGenerated > ago(timeOffset*2) and TimeGenerated < ago(timeOffset) 
| where EventID != discardEventId

## 'let' statement: declare dynamic tables or lists

In [ ]:
%%kql
let suspiciousAccounts = datatable(account: string) [ 
    @"\administrator",
    @"NT AUTHORITY\SYSTEM"];
SecurityEvent 
| where Account in (suspiciousAccounts) 

In [ ]:
%%kql
let LowActivityAccounts = SecurityEvent
|summarize cnt = count() by Account
| where cnt <10;
//
LowActivityAccounts
| where Account contains "Mal"

## 'materialize' statement

In [ ]:
%%kql
let LowActivityAccounts = materialize(SecurityEvent
                                        | summarize cnt = count() by Account | where cnt < 10);
LowActivityAccounts
| where Account contains "Mal"
| union (LowActivityAccounts | where Account contains "Rep")

## 'union' operator

In [ ]:
%%kql
SecurityEvent
| union (WindowsFirewall | where CommunicationDirection == "RECEIVE")

## Lab #6: union
Find the ratio of alerts (in the SecurityAlert table) to events (in the SecurityEvent table) broken by day for the last week

In [ ]:
%%kql
SecurityAlert 
| union SecurityEvent 
| summarize SecurityAlerts = countif(Type == "SecurityAlert"), 
            SecurityEvents = countif(Type == "SecurityEvent") by bin (TimeGenerated, 24h)
| extend Ratio = SecurityAlerts * 1.0 / SecurityEvents 
| project SecurityEvents , SecurityAlerts , Ratio

## 'join' operator

In [ ]:
%%kql
SecurityEvent
|join (SecurityAlert | where AlertSeverity == 'High'| extend Account = tostring(parse_json(ExtendedProperties).["User Name"])
) on Account

## JSON exercise

In [ ]:
%%kql
SecurityAlert
| extend ExtendedProperties = todynamic(ExtendedProperties) 
| extend ActionTaken = ExtendedProperties.ActionTaken
| extend AttackerIP = ExtendedProperties["Attacker IP"]

In [ ]:
%%kql 
SecurityAlert
| mv-expand entity = todynamic(Entities)

In [ ]:
%%kql
SecurityAlert
| mv-apply entity = todynamic(Entities) on (
    where entity.Type == "account"
    | extend account = strcat (entity.NTDomain, "\\", entity.Name))

## Lab #7
Show for each account that has alerts, how many alerts and which Security Event types it had in the last 7 days.

Hints and guideline:
- Alerts in the SecurityAlert table keeps entities in a JSON array string. What is the name of this field?
- Account is just one of the possible entity types in the array
- Make sure that account has the same format in both the alerts and events table.

In [ ]:
%%kql
SecurityAlert
| mv-apply entity=todynamic(Entities) on (
    where entity.Type == "account"
    | extend Account = strcat(entity.NTDomain, "\\", entity.Name))
| summarize dcount(SystemAlertId) by Account
| join kind=leftouter ( SecurityEvent
| summarize make_set(EventID) by Account ) on Account